### `LSTM.py`

In [ ]:
!pip install music21

In [1]:
import glob
import pickle
import numpy
import keras.utils
from music21 import converter, instrument, note, chord, stream, duration
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint

2024-04-29 20:19:13.247874: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 20:19:14.197023: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 20:19:14.197080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 20:19:14.382602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 20:19:14.794320: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_files = glob.glob('pijama_jazz/*.midi')

In [4]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()
    # notes = pickle.load('/content/drive/My Drive/Colab Notebooks/final_project_neuro240_files/data/notes')

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [5]:
len(train_files)

26

In [6]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in train_files[:10]:
        midi = converter.parse(file)

        print("Parsing %s" % file)

        for transposition in range(-6, 6):  # Transpose from -6 to +5 semitones
            transposed_midi = midi.transpose(transposition)
            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(transposed_midi)
                notes_to_parse = s2.parts[0].recurse()
            except: # file has notes in a flat structure
                notes_to_parse = transposed_midi.flat.notes
    
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    # print(str(element.pitch) + "_" + str(element.duration.quarterLength))
                    notes.append(str(element.pitch) + "_" + str(element.duration.quarterLength))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
                elif isinstance(element, note.Rest):
                    notes.append("R")
    # try:
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    print("Notes saved")
    # except:
    #     pass

    return notes

In [7]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = keras.utils.to_categorical(network_output)

    return (network_input, network_output)

In [8]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [13]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "Jazz-Rest-Duration-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

In [14]:
train_network()

Parsing pijama_jazz/Good Citizen.midi
Parsing pijama_jazz/The Flower Song.midi
Parsing pijama_jazz/The Wedding.midi
Parsing pijama_jazz/Crawdaddy.midi
Parsing pijama_jazz/Gay Divorce -  After You, Who.midi
Parsing pijama_jazz/Easy to Love.midi
Parsing pijama_jazz/I've Got You Under My Skin.midi
Parsing pijama_jazz/Beautiful Dreamer.midi
Parsing pijama_jazz/A Rose for Emily.midi
Parsing pijama_jazz/Here, There and Everywhere.midi
Notes saved
Epoch 1/200
1618/1618 [==============================] - ETA: 0s - loss: 5.5082

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1618/1618 [==============================] - 661s 404ms/step - loss: 5.5082
Epoch 2/200
1618/1618 [==============================] - 653s 404ms/step - loss: 5.1520
Epoch 3/200
1618/1618 [==============================] - 654s 404ms/step - loss: 5.1262
Epoch 4/200
1618/1618 [==============================] - 654s 404ms/step - loss: 5.1170
Epoch 5/200
1618/1618 [==============================] - 664s 410ms/step - loss: 5.1149
Epoch 6/200
1618/1618 [==============================] - 665s 411ms/step - loss: 5.1042
Epoch 7/200
1618/1618 [==============================] - 666s 411ms/step - loss: 5.0931
Epoch 8/200
1618/1618 [==============================] - 666s 411ms/step - loss: 5.0812
Epoch 9/200
1618/1618 [==============================] - 665s 411ms/step - loss: 5.0715
Epoch 10/200
1618/1618 [==============================] - 665s 411ms/step - loss: 5.0579
Epoch 11/200
1618/1618 [==============================] - 663s 410ms/step - loss: 5.0443
Epoch 12/200
1618/1618 [==================

KeyboardInterrupt: 

### `predict.py`

In [3]:
WEIGHTS_PATH = "Jazz-Rest-Duration-weights-improvement-76-3.1108.hdf5"

In [4]:
def adjust_rest_probability(predictions, rest_idx, penalty_factor=0.5):
    """Reduce the probability of predicting a rest by a penalty factor."""
    predictions[0, rest_idx] *= penalty_factor
    predictions /= numpy.sum(predictions)  # Re-normalize probabilities
    return predictions

In [5]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [6]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [7]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(WEIGHTS_PATH)

    return model

In [8]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    rest_index = note_to_int.get('R', None)  # Safely get rest index, if it exists

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        # Adjust probability for rests, if rest index is known
        if rest_index is not None:
            prediction = adjust_rest_probability(prediction, rest_index, penalty_factor=0.1)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [11]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                try:
                    new_note = note.Note(current_note.split('_')[0])
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                except:
                    try:
                        new_note = note.Note(current_note)
                        new_note.storedInstrument = instrument.Piano()
                        notes.append(new_note)
                    except:
                        try:
                            new_note = note.Note(int(current_note))
                            new_note.storedInstrument = instrument.Piano()
                            notes.append(new_note)
                        except:
                            pass
            try:
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            except:
                pass
        # pattern is a rest
        elif pattern == "R":
            # print("rest detected")
            new_rest = note.Rest()
            # new_rest.duration.quarterLength = 0.5
            new_rest.storedInstrument = instrument.Piano()
            new_rest.offset = offset
            output_notes.append(new_rest)
        # pattern is a note
        else:
            # print(pattern)
            note_pitch, note_length = pattern.split("_")
            # print((note_pitch, note_length))
            try:
                new_note = note.Note(note_pitch)
                try:
                    new_note.duration = duration.Duration(float(note_length))
                    new_note.storedInstrument = instrument.Piano()
                    new_note.offset = offset
                    output_notes.append(new_note)
                except:
                    new_note.storedInstrument = instrument.Piano()
                    new_note.offset = offset
                    output_notes.append(new_note)
            except:
                pass

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_jazz1.mid')

In [12]:
generate()

2024-04-29 20:20:27.151376: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
